#  Visualize Results

In [1]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.code import Code
from aiida.orm import load_node
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.base import Int, Float, Str, Bool
from aiida.work.run import submit
from aiida.orm.data.structure import StructureData

import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
from ase import Atoms
from ase.io import read,write
from ase.visualize import view
from ase.data import vdw_radii


from apps.surfaces.widgets import find_mol
# Phonon specific import
from apps.phonons.widgets.structure_browser import StructureBrowser
from apps.phonons.widgets.viewer_details import ViewerDetails






In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
## GENERAL PARAMETERS

In [4]:
## WIDGETS MONITOR FUNCTIONS
def on_mode_change(c):
    modes = struct_browser.modes.value
    if not modes == 'False':
        node = struct_browser.results.value
        file_path_mol = node.out.retrieved.folder.abspath + "/path/aiida-VIBRATIONS-1.mol"
        file_path_xyz = node.inp.molslab_coords.get_file_abs_path()
        atoms = read(file_path_xyz)
        slab_analyzed = find_mol.analyze_slab(atoms)
        viewer_widget.setup(file_path_mol,atoms,modes, details=slab_analyzed)
    
def set_modes(c):
    file_path = struct_browser.results.value.out.retrieved.folder.abspath + "/path/aiida-VIBRATIONS-1.mol"
    for line in reversed(open(file_path).readlines()):
        if "vibration" in line:
            break
    line_array = [int(s) for s in line.split() if s.isdigit()]
    last_mode = line_array[0]
    
    struct_browser.set_modes(max_modes=last_mode)





SyntaxError: invalid syntax (<ipython-input-4-fe021f6f8a9e>, line 6)

In [ ]:
## DISPLAY WIDGETS AND DEFINE JOB PARAMETERS

##STRUCTURE
struct_browser = StructureBrowser()
struct_browser.results.observe(set_modes, names='value') 
struct_browser.modes.observe(on_mode_change, names='value') 


##VIEWER
viewer_widget = ViewerDetails()
display(ipw.VBox([struct_browser, viewer_widget]))


